# INTRODUCTION TO DES simulation with simpy

Simpy is a Python library to design process-based discrete-event simulation systems. Detailed documentation can be found here:

https://pypi.org/project/simpy/




In [1]:
pip install simpy

## Generator functions vs regular python functions

The Python instruction *yield* is one of the key ingredients in the simpy libraries, since it allows to suspend the process during a certain time period until the function is executed again. This is rather convenient when performing simulations of discrete-event systems. More information about Python generators can be found here: https://wiki.python.org/moin/Generators

Let's start by a short expample to show the differences between regular functions and generator functions. 

In [2]:
def mygen(n): 
    while True: 
      for j in range(10):
        yield j
def myfun(n): 
    while True: 
      for j in range(10):
        return j

In a regular function, the output will always be zero since it returns the first value of the for iterator:


In [4]:
myfun(10)

0

In [5]:
myfun(10)

0

In a generator function, local variables and theirs states are remembered between successive calls:

In [6]:
out1 = mygen(10)

In [7]:
out1

<generator object mygen at 0x7fd3c8a16fc0>

In [8]:
next(out1)


0

In [9]:
next(out1)


1

In [11]:
next(out1)


3

# Some simpy introductory examples:

## Example 1: Traffic light: 
The first example is a basic loop simulating a traffic light that is green for 25 seconds, yellow during 5 seconds and red for 60 seconds. By executing the simulation we can see at wehich times the trafficl light changes its state:

In [12]:
import simpy

# Generator function that defines the working of the traffic light  
# "timeout()" function makes next yield statement wait for a  
# given time passed as the argument  
def Traffic_Light(env):  
  
    while True:  
  
        print ("Light turns GRN at " + str(env.now))  
          
        # Light is green for 25 seconds  
        yield env.timeout(25)         
  
        print ("Light turns YEL at " + str(env.now)) 
          
        # Light is yellow for 5 seconds  
        yield env.timeout(5)  
  
        print ("Light turns RED at " + str(env.now))  
          
        # Light is red for 60 seconds  
        yield env.timeout(60)  
  
# env is the environment variable  
env = simpy.Environment()         
  
# The process defined by the function Traffic_Light(env)  
# is added to the environment  
env.process(Traffic_Light(env))  
  
# The process is run for the first 180 seconds (180 is not included)  
env.run(until = 180)  

Light turns GRN at 0
Light turns YEL at 25
Light turns RED at 30
Light turns GRN at 90
Light turns YEL at 115
Light turns RED at 120


## Example 2: Parking lot

In this case, the simulation of a parking lot 

In [ ]:
def car(env):
  while True:
    print('Start parking at %d' % env.now)
    parking_duration = 5
    yield env.timeout(parking_duration)

    print('Start driving at %d' % env.now)
    trip_duration = 2
    yield env.timeout(trip_duration)
    
env = simpy.Environment()
env.process(car(env))
env.run(until=60)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14
Start driving at 19
Start parking at 21
Start driving at 26
Start parking at 28
Start driving at 33
Start parking at 35
Start driving at 40
Start parking at 42
Start driving at 47
Start parking at 49
Start driving at 54
Start parking at 56


## Example 3: Two clocks

Adapted from: https://simpy.readthedocs.io/en/latest/index.html

One fast, one slow

In [13]:
import simpy

def clock(env, name, tick):
  while True:
    print(name, env.now)
    yield env.timeout(tick)

env = simpy.Environment()
env.process(clock(env, 'fast', 0.5))
env.process(clock(env, 'slow', 1))
env.run(until=10)

fast 0
slow 0
fast 0.5
slow 1
fast 1.0
fast 1.5
slow 2
fast 2.0
fast 2.5
slow 3
fast 3.0
fast 3.5
slow 4
fast 4.0
fast 4.5
slow 5
fast 5.0
fast 5.5
slow 6
fast 6.0
fast 6.5
slow 7
fast 7.0
fast 7.5
slow 8
fast 8.0
fast 8.5
slow 9
fast 9.0
fast 9.5


## Example 4: A simple queue system

Adapted from: https://simpy.readthedocs.io/en/latest/examples/bank_renege.html?highlight=random

A total of **NEW_CUSTOMERS** customers arrive randomly following an exponential arrival time with a mean Time Between Arrivals specified by parameter **INTERVAL_CUSTOMERS**. 

The code defines a generator function called *source* to randomly generate the customers, and a generator function *customer* in which the customers are served. Customers can renege and leave when waiting a certain random time in the interval **[MIN_PATIENCE, MAX_PATIENCE]**




In [ ]:
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # Total number of customers
INTERVAL_CUSTOMERS = 3.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))

# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.0602 Customer01: Here I am
 3.8179 Customer02: Here I am
 3.8595 Customer00: Finished
 3.8595 Customer01: Waited  0.799
 4.9509 Customer01: Finished
 4.9509 Customer02: Waited  1.133
 7.2055 Customer03: Here I am
 7.2962 Customer04: Here I am
 8.3493 Customer04: RENEGED after  1.053
 8.6427 Customer03: RENEGED after  1.437
 9.4080 Customer05: Here I am
10.0730 Customer06: Here I am
11.5139 Customer06: RENEGED after  1.441
11.5275 Customer02: Finished
11.5275 Customer05: Waited  2.120
12.4350 Customer07: Here I am
13.4480 Customer07: RENEGED after  1.013
17.4082 Customer08: Here I am
19.8045 Customer08: RENEGED after  2.396
22.2052 Customer05: Finished
22.3251 Customer09: Here I am
22.3251 Customer09: Waited  0.000
60.1434 Customer09: Finished
